# Portfolio Analysis Dashboard

포트폴리오 분석 및 시각화

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sqlalchemy import create_engine, text
from datetime import date, datetime, timedelta
import sys
sys.path.insert(0, '..')

from config.settings import Settings

settings = Settings()

# DB 연결 (터널링 시 포트 변경 필요)
DB_HOST = "127.0.0.1"
DB_PORT = 3306  # 터널링 시 변경 (예: 3307)

engine = create_engine(
    f"mysql+pymysql://{settings.DB_USER}:{settings.DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{settings.DB_NAME}?charset=utf8mb4"
)

# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

print(f"Connected to {settings.DB_NAME}")

In [ ]:
# 유틸리티 함수: 가장 최근 거래일 찾기
def get_latest_trading_date(engine, target_date=None):
    """주어진 날짜 이하의 가장 최근 거래일 반환"""
    if target_date is None:
        target_date = date.today()
    
    sql = """
    SELECT MAX(snapshot_date) AS latest_date
    FROM daily_portfolio_snapshot
    WHERE snapshot_date <= :target_date
    """
    result = pd.read_sql(text(sql), engine, params={"target_date": target_date})
    return result.loc[0, 'latest_date']

# 가장 최근 거래일
LATEST_TRADING_DATE = get_latest_trading_date(engine)
print(f"가장 최근 거래일: {LATEST_TRADING_DATE}")

---
## 1. 오늘 매도한 종목 (실현수익률, 보유기간)

In [ ]:
# 매도일 선택 (기본: 가장 최근 거래일)
SELL_DATE = LATEST_TRADING_DATE  # 또는 date(2026, 1, 30)

sql_sold = """
SELECT
    closed_date AS 매도일,
    stock_name AS 종목명,
    CAST(avg_purchase_price AS SIGNED) AS 매수가,
    CAST(current_price AS SIGNED) AS 매도가,
    ROUND(realized_pnl / total_cost * 100, 2) AS 실현수익률,
    holding_days AS 홀딩기간,
    CASE 
        WHEN EXISTS (
            SELECT 1 FROM daily_lots dl2 
            WHERE dl2.stock_code = daily_lots.stock_code 
            AND dl2.is_closed = FALSE
        ) THEN 'Y' 
        ELSE 'N' 
    END AS 현재보유여부
FROM daily_lots
WHERE is_closed = TRUE
  AND closed_date = :sell_date
ORDER BY 실현수익률 DESC
"""

df_sold = pd.read_sql(text(sql_sold), engine, params={"sell_date": SELL_DATE})

if len(df_sold) == 0:
    print(f"{SELL_DATE}에 매도한 종목이 없습니다.")
else:
    print(f"{SELL_DATE} 매도 종목: {len(df_sold)}건")
    
    # 스타일링
    df_styled = (
        df_sold.style
        .format({
            "매수가": "{:,.0f}",
            "매도가": "{:,.0f}",
            "실현수익률": "{:+.2f}%",
            "홀딩기간": "{:d}일"
        })
        .background_gradient(subset=["실현수익률"], cmap="RdYlGn", vmin=-10, vmax=50)
        .set_properties(subset=["현재보유여부"], **{"text-align": "center"})
    )
    display(df_styled)

In [ ]:
# 매도 종목 테이블 이미지로 저장 (선택사항)
if len(df_sold) > 0:
    try:
        import dataframe_image as dfi
        dfi.export(df_styled, f"sold_{SELL_DATE}.png", table_conversion="matplotlib", dpi=200)
        print(f"Saved: sold_{SELL_DATE}.png")
    except ImportError:
        print("dataframe_image not installed. Run: pip install dataframe_image")

---
## 2. 포트폴리오 보유 비중 (파이 차트)

In [ ]:
sql_weight = """
SELECT
    stock_name AS stk_nm,
    SUM(market_value) AS market_value,
    SUM(portfolio_weight_pct) AS weight_pct
FROM portfolio_snapshot
WHERE snapshot_date = :snapshot_date
GROUP BY stock_code, stock_name
ORDER BY market_value DESC
"""

sql_exposure = """
SELECT
    aset_evlt_amt,
    prsm_dpst_aset_amt,
    ROUND(aset_evlt_amt / prsm_dpst_aset_amt * 100, 1) AS stock_exposure_pct
FROM account_summary
WHERE snapshot_date = :snapshot_date
"""

df_weight = pd.read_sql(text(sql_weight), engine, params={"snapshot_date": LATEST_TRADING_DATE})
df_exposure = pd.read_sql(text(sql_exposure), engine, params={"snapshot_date": LATEST_TRADING_DATE})

stock_exposure_pct = df_exposure.loc[0, "stock_exposure_pct"] if len(df_exposure) > 0 else 0
total_asset = df_exposure.loc[0, "prsm_dpst_aset_amt"] if len(df_exposure) > 0 else 0  # 추정자산

print(f"총 {len(df_weight)}개 종목")
print(f"추정자산(청산기준): {total_asset:,.0f}원")
print(f"자산대비 주식비중: {stock_exposure_pct:.1f}%")

In [ ]:
# 파이 차트 설정
TOP_N = len(df_weight)  # 전체 종목 표시 (필요시 숫자로 변경: 예) TOP_N = 20)

df_plot = df_weight.copy()
df_plot["market_value"] = pd.to_numeric(df_plot["market_value"], errors="coerce").fillna(0)
df_plot = df_plot[df_plot["market_value"] > 0].sort_values("market_value", ascending=False)

# TOP N + 기타
if len(df_plot) > TOP_N:
    top = df_plot.head(TOP_N).copy()
    other_sum = df_plot.iloc[TOP_N:]["market_value"].sum()
    if other_sum > 0:
        top = pd.concat([top, pd.DataFrame([{"stk_nm": "기타", "market_value": other_sum}])], ignore_index=True)
    df_plot = top

# 추정자산 기준 비중 계산
df_plot["pct"] = df_plot["market_value"] / total_asset * 100

print(f"차트에 표시할 종목 수: {len(df_plot)}")

In [ ]:
# 파이 차트 그리기
fig = plt.figure(figsize=(14, 9))
ax_pie = fig.add_axes([0.02, 0.05, 0.70, 0.90])
ax_text = fig.add_axes([0.70, 0.30, 0.25, 0.40])

wedges, texts, autotexts = ax_pie.pie(
    df_plot["market_value"],
    labels=df_plot["stk_nm"],
    autopct=lambda p: f"{p:.1f}%",
    startangle=90,
    counterclock=False,
    radius=1.2,
    labeldistance=1.05,
    pctdistance=0.75
)

ax_pie.set_title("포트폴리오 보유 비중 (평가금액 기준)", fontsize=18, pad=26)

for t in texts:
    t.set_fontsize(11)
for t in autotexts:
    t.set_fontsize(10)

# 우측 텍스트
ax_text.axis("off")
ax_text.text(0.25, 0.65, "자산 대비 주식 비중", fontsize=14, fontweight="bold")
ax_text.text(0.25, 0.50, f"{stock_exposure_pct:.1f}%", fontsize=22, fontweight="bold")
ax_text.text(0.25, 0.25, "※ 정의\n주식 평가금액 ÷\n청산 기준 총자산\n(레버리지 포함)", fontsize=11, color="gray")

plt.show()

---
## 3. 보유 종목별 수익률 / 홀딩기간 (막대 차트)

In [ ]:
sql_holdings = """
SELECT
    stock_name AS 종목명,
    trade_date AS 매수일,
    CAST(avg_purchase_price AS SIGNED) AS 매수평균가,
    ROUND(unrealized_return_pct, 2) AS 수익률,
    holding_days AS 홀딩기간
FROM daily_lots
WHERE is_closed = FALSE
ORDER BY unrealized_return_pct DESC
"""

df_holdings = pd.read_sql(text(sql_holdings), engine)
print(f"보유 종목 (lot 기준): {len(df_holdings)}개")
df_holdings.head(10)

In [ ]:
# 막대 차트 설정
MAX_BARS = len(df_holdings)  # 전체 표시 (필요시 숫자로 변경: 예) MAX_BARS = 35)

plot_df = df_holdings.head(MAX_BARS).copy()

x = np.arange(len(plot_df))
y = plot_df["수익률"].astype(float).values
holding = plot_df["홀딩기간"].astype(int).values
names = plot_df["종목명"].astype(str).values

plt.figure(figsize=(max(14, len(plot_df) * 0.4), 6))

colors = ["#f08060" if v >= 0 else "#4f7bd9" for v in y]
bars = plt.bar(x, y, color=colors, edgecolor="black", linewidth=0.4)

plt.axhline(0, color="gray", linewidth=0.8)
plt.xticks(x, names, rotation=45, ha="right", fontsize=9)
plt.ylabel("수익률 (%)")
plt.title("보유 종목별 수익률(%) / 홀딩기간", fontsize=14)

ymax = max(y.max(), 0)
ymin = min(y.min(), 0)
pad = max((ymax - ymin) * 0.25, 4)
plt.ylim(ymin - pad, ymax + pad)

INSIDE_THRESHOLD = 4.0
OFFSET = 1.8

for bar, ret, days in zip(bars, y, holding):
    cx = bar.get_x() + bar.get_width() / 2
    
    # 홀딩기간 (바깥)
    holding_y = ret + (OFFSET if ret >= 0 else -OFFSET)
    plt.text(cx, holding_y, f"{days}일", ha="center", 
             va="bottom" if ret >= 0 else "top", fontsize=8, color="black")
    
    # 수익률
    if abs(ret) >= INSIDE_THRESHOLD:
        ret_y = ret * 0.5
        va = "center"
    else:
        ret_y = ret - OFFSET if ret >= 0 else ret + OFFSET
        va = "top" if ret >= 0 else "bottom"
    
    plt.text(cx, ret_y, f"{ret:+.2f}%", ha="center", va=va, 
             fontsize=9, color="black", fontweight="bold")

plt.tight_layout()
plt.show()

---
## 4. 포트폴리오 vs KOSPI/KOSDAQ 수익률 비교

In [ ]:
# 기간 설정
START_DATE = date(2025, 12, 11)
END_DATE = LATEST_TRADING_DATE

sql_portfolio = """
SELECT
    snapshot_date,
    day_stk_asst AS total_asset,
    COALESCE(ina_amt, 0) AS deposit,
    COALESCE(outa, 0) AS withdrawal
FROM daily_portfolio_snapshot
WHERE snapshot_date BETWEEN :start_date AND :end_date
ORDER BY snapshot_date
"""

sql_index = """
SELECT
    index_date,
    kospi_close,
    kosdaq_close
FROM market_index
WHERE index_date BETWEEN :start_date AND :end_date
ORDER BY index_date
"""

df_portfolio = pd.read_sql(text(sql_portfolio), engine, params={"start_date": START_DATE, "end_date": END_DATE})
df_index = pd.read_sql(text(sql_index), engine, params={"start_date": START_DATE, "end_date": END_DATE})

print(f"Portfolio data: {len(df_portfolio)} days")
print(f"Index data: {len(df_index)} days")

In [ ]:
# 시간가중수익률 (TWR) 계산 - 일별 누적 포함
def calculate_twr_daily(df):
    """Time-Weighted Return 계산 (일별 누적 수익률 반환)"""
    if len(df) < 2:
        return pd.DataFrame(), 0.0
    
    df = df.sort_values('snapshot_date').reset_index(drop=True)
    cumulative_returns = [1.0]  # 시작점
    
    for i in range(1, len(df)):
        prev_asset = df.loc[i-1, 'total_asset']
        curr_asset = df.loc[i, 'total_asset']
        cash_flow = df.loc[i, 'deposit'] - df.loc[i, 'withdrawal']
        
        # 현금흐름 조정: 기초자산에 현금흐름 더함
        adjusted_prev = prev_asset + cash_flow
        
        if adjusted_prev > 0:
            period_return = curr_asset / adjusted_prev
        else:
            period_return = 1.0
        
        cumulative_returns.append(cumulative_returns[-1] * period_return)
    
    df_result = df[['snapshot_date']].copy()
    df_result['twr_cumulative'] = cumulative_returns
    df_result['twr_pct'] = (df_result['twr_cumulative'] - 1) * 100
    
    final_twr = (cumulative_returns[-1] - 1) * 100
    return df_result, final_twr

# 금액가중수익률 (MWR) 계산 - 일별 근사
def calculate_mwr_daily(df):
    """Money-Weighted Return 일별 근사 계산"""
    if len(df) < 2:
        return pd.DataFrame(), 0.0
    
    df = df.sort_values('snapshot_date').reset_index(drop=True)
    
    initial_asset = df.loc[0, 'total_asset']
    results = []
    
    cumulative_deposit = 0
    cumulative_withdrawal = 0
    
    for i in range(len(df)):
        cumulative_deposit += df.loc[i, 'deposit']
        cumulative_withdrawal += df.loc[i, 'withdrawal']
        
        current_asset = df.loc[i, 'total_asset']
        net_cash_flow = cumulative_deposit - cumulative_withdrawal
        
        # MWR 근사: (현재자산 - 초기자산 - 순입금) / (초기자산 + 순입금/2)
        gain = current_asset - initial_asset - net_cash_flow
        avg_capital = initial_asset + net_cash_flow / 2
        
        if avg_capital > 0:
            mwr_pct = (gain / avg_capital) * 100
        else:
            mwr_pct = 0
        
        results.append({
            'snapshot_date': df.loc[i, 'snapshot_date'],
            'mwr_pct': mwr_pct
        })
    
    df_result = pd.DataFrame(results)
    final_mwr = results[-1]['mwr_pct'] if results else 0.0
    return df_result, final_mwr

# 지수 수익률 계산 - 일별
def calculate_index_return_daily(df, column):
    """지수 일별 수익률 계산"""
    if len(df) < 1:
        return pd.DataFrame(), 0.0
    
    df = df.sort_values('index_date').reset_index(drop=True)
    start_val = df.loc[0, column]
    
    df_result = df[['index_date']].copy()
    df_result[f'{column}_pct'] = (df[column] / start_val - 1) * 100
    
    final_return = df_result[f'{column}_pct'].iloc[-1]
    return df_result, final_return

In [ ]:
# 수익률 계산
df_twr, twr = calculate_twr_daily(df_portfolio)
df_mwr, mwr = calculate_mwr_daily(df_portfolio)
df_kospi, kospi_return = calculate_index_return_daily(df_index, 'kospi_close')
df_kosdaq, kosdaq_return = calculate_index_return_daily(df_index, 'kosdaq_close')

print(f"\n=== 수익률 비교 ({START_DATE} ~ {END_DATE}) ===")
print(f"")
print(f"내 포트폴리오 (TWR): {twr:+.2f}%")
print(f"내 포트폴리오 (MWR): {mwr:+.2f}%")
print(f"KOSPI:              {kospi_return:+.2f}%")
print(f"KOSDAQ:             {kosdaq_return:+.2f}%")
print(f"")
print(f"vs KOSPI 초과수익:  {twr - kospi_return:+.2f}%p")
print(f"vs KOSDAQ 초과수익: {twr - kosdaq_return:+.2f}%p")

In [ ]:
# 비교 막대 차트
returns = {
    '내 포트폴리오\n(TWR)': twr,
    '내 포트폴리오\n(MWR)': mwr,
    'KOSPI': kospi_return,
    'KOSDAQ': kosdaq_return
}

fig, ax = plt.subplots(figsize=(10, 6))

x = np.arange(len(returns))
values = list(returns.values())
colors = ['#2ecc71' if v >= 0 else '#e74c3c' for v in values]

bars = ax.bar(x, values, color=colors, edgecolor='black', linewidth=0.5, width=0.6)

ax.axhline(0, color='gray', linewidth=0.8)
ax.set_xticks(x)
ax.set_xticklabels(returns.keys(), fontsize=12)
ax.set_ylabel('수익률 (%)', fontsize=12)
ax.set_title(f'수익률 비교 ({START_DATE} ~ {END_DATE})', fontsize=14, fontweight='bold')

# 값 표시
for bar, val in zip(bars, values):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2, height + (1 if height >= 0 else -2),
            f'{val:+.2f}%', ha='center', va='bottom' if height >= 0 else 'top',
            fontsize=14, fontweight='bold')

# y축 범위 조정
ymax = max(max(values), 0) + 5
ymin = min(min(values), 0) - 5
ax.set_ylim(ymin, ymax)

plt.tight_layout()
plt.show()

In [ ]:
# 일별 누적 수익률 추이 (TWR 기준 - 입출금 영향 제거)

# 데이터 병합
df_merged = pd.merge(
    df_twr[['snapshot_date', 'twr_pct']],
    df_kospi[['index_date', 'kospi_close_pct']],
    left_on='snapshot_date',
    right_on='index_date',
    how='inner'
)

df_merged = pd.merge(
    df_merged,
    df_kosdaq[['index_date', 'kosdaq_close_pct']],
    on='index_date',
    how='inner'
).sort_values('snapshot_date')

if len(df_merged) > 0:
    fig, ax = plt.subplots(figsize=(12, 6))
    
    ax.plot(df_merged['snapshot_date'], df_merged['twr_pct'], 
            label='내 포트폴리오 (TWR)', linewidth=2.5, color='#3498db')
    ax.plot(df_merged['snapshot_date'], df_merged['kospi_close_pct'], 
            label='KOSPI', linewidth=1.5, color='#e74c3c', linestyle='--')
    ax.plot(df_merged['snapshot_date'], df_merged['kosdaq_close_pct'], 
            label='KOSDAQ', linewidth=1.5, color='#2ecc71', linestyle='--')
    
    ax.axhline(0, color='gray', linewidth=0.5, linestyle=':')
    ax.fill_between(df_merged['snapshot_date'], 0, df_merged['twr_pct'], 
                    alpha=0.1, color='#3498db')
    
    ax.set_xlabel('날짜')
    ax.set_ylabel('누적 수익률 (%)')
    ax.set_title('포트폴리오 vs 시장지수 누적 수익률 (TWR 기준)', fontsize=14, fontweight='bold')
    ax.legend(loc='upper left')
    ax.grid(True, alpha=0.3)
    
    # 최종 수익률 표시
    last_date = df_merged['snapshot_date'].iloc[-1]
    ax.annotate(f'{twr:+.2f}%', xy=(last_date, twr), fontsize=10, fontweight='bold', color='#3498db')
    ax.annotate(f'{kospi_return:+.2f}%', xy=(last_date, kospi_return), fontsize=9, color='#e74c3c')
    ax.annotate(f'{kosdaq_return:+.2f}%', xy=(last_date, kosdaq_return), fontsize=9, color='#2ecc71')
    
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
else:
    print("데이터가 부족합니다.")

---
## 5. 요약 테이블

In [ ]:
# 전체 요약
summary_data = {
    '지표': [
        f'기간',
        '내 포트폴리오 (TWR)',
        '내 포트폴리오 (MWR)',
        'KOSPI',
        'KOSDAQ',
        'vs KOSPI 초과수익',
        'vs KOSDAQ 초과수익'
    ],
    '값': [
        f'{START_DATE} ~ {END_DATE} ({(END_DATE - START_DATE).days}일)',
        f'{twr:+.2f}%',
        f'{mwr:+.2f}%',
        f'{kospi_return:+.2f}%',
        f'{kosdaq_return:+.2f}%',
        f'{twr - kospi_return:+.2f}%p',
        f'{twr - kosdaq_return:+.2f}%p'
    ]
}

df_summary = pd.DataFrame(summary_data)
df_summary.style.set_properties(**{'text-align': 'right'}).set_properties(subset=['지표'], **{'text-align': 'left', 'font-weight': 'bold'})

In [ ]:
# DB 연결 종료
engine.dispose()
print("Done!")